At first: 
* wanted to identify factors associated with the actual outcomes ('warning', 'citation', 'arrest') 
* but a quick glimpse of the raw dataset, I realized this inference analysis might be boring since most warnings and citations are related to either moving or mechanical/non-moving "reason_for_stop" 
* i.e. it would seem like "reason_for_stop" almost always determine the "outcome," so there wouldn't really be a point in looking at other factors (gender, age, etc) as "predictors"

Now: 
* another column I want to analyze more closely was whether or not a search was conducted ("search_conducted")
* after glancing at some of the corresponding values in "reason_for_stop," I see variety in the reasons for stops
    * e.g for a moving violation or mechanical/non-moving etc, one will get searched vs another (perhaps due to human bias etc)
* so this column seems to have more promising/interesting results (hopefully)

---

## Overview of Plan 

**Guiding Question for Doing EDA (purpose):**

After accounting for the legal reason for the stop (moving violation, DUI, etc), *what other factors (gender, location,hour of day, etc) can statistically explain the choice to conduct a search?*

e.g: 
* I will control for the "reason_for_stop" by grouping, then let's say I did some analysis to see if gender is associated with getting a search or not
* I find that male has a higher search rate compared to females in the group "moving violation" 
* ==> even when the reason for being stopped is the same, analysis shows that gender is associated w/ higher likelihood of being searched

**Rough Workflow**

Recall specs:
1. Data Overview
    * Load the dataset.
    * Summarize rows, columns, variable types, missing values, and duplicates.
2. Descriptive Statistics
    * Calculate summary metrics (mean, median, standard deviation, min, max, counts).
3. Visual Exploration
    * Create histograms, boxplots, scatterplots, and a correlation heatmap.
4. Data Quality Review
    * Identify missing data, outliers, and unusual values.
5. Model-Relevant Insights
    * Identify which variables may be useful predictors.
    * Provide a plain-language summary of takeaways.

* instead of deeply analyzing every column (i.e following step-by-step specs above ^) to identify the best features/predictors, I will pick 4-5 features after briefly analyzing all the columns w/ steps 1, 2, 4
* from the chosen features, I will explore them visually to explore their distribution and identify possible relationships with being searched (step 3, 4)
___

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
column_types = {'district': str}
sf_stops_df = pd.read_csv("ca_san_francisco_2020_04_01.csv", dtype = column_types)
sf_stops_df.head()

,raw_row_number,date,time,location,lat,lng,district,subject_age,subject_race,subject_sex,...,citation_issued,warning_issued,outcome,contraband_found,search_conducted,search_vehicle,search_basis,reason_for_stop,raw_search_vehicle_description,raw_result_of_contact_description
0,869921,2014-08-01,00:01:00,MASONIC AV & FELL ST,37.773004,-122.445873,NaN,NaN,asian/pacific islander,female,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning
1,869922,2014-08-01,00:01:00,GEARY&10TH AV,37.780898,-122.468586,NaN,NaN,black,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
2,869923,2014-08-01,00:15:00,SUTTER N OCTAVIA ST,37.786919,-122.426718,NaN,NaN,hispanic,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
3,869924,2014-08-01,00:18:00,3RD ST & DAVIDSON,37.746380,-122.392005,NaN,NaN,hispanic,male,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning
4,869925,2014-08-01,00:19:00,DIVISADERO ST. & BUSH ST.,37.786348,-122.440003,NaN,NaN,white,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation


___
## Data Overview, Descriptive Stats, Data Quality Review 
(Steps 1, 2, 4)

In [59]:
display(sf_stops_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905070 entries, 0 to 905069
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   raw_row_number                     905070 non-null  object 
 1   date                               905070 non-null  object 
 2   time                               905035 non-null  object 
 3   location                           905027 non-null  object 
 4   lat                                903373 non-null  float64
 5   lng                                903373 non-null  float64
 6   district                           852883 non-null  object 
 7   subject_age                        846182 non-null  float64
 8   subject_race                       905070 non-null  object 
 9   subject_sex                        905070 non-null  object 
 10  type                               905070 non-null  object 
 11  arrest_made                        9050

None

In [57]:
print("Shape of df:", sf_stops_df.shape, "\n")
print("Duplicates:", sf_stops_df.duplicated().sum(), "\n")
print("Missing Data:")
print(sf_stops_df.isna().sum())


Shape of df: (905070, 22) 

Duplicates: 0 

Missing Data:
raw_row_number                            0
date                                      0
time                                     35
location                                 43
lat                                    1697
lng                                    1697
district                              52187
subject_age                           58888
subject_race                              0
subject_sex                               0
type                                      0
arrest_made                               0
citation_issued                           0
warning_issued                            0
outcome                               15681
contraband_found                     851689
search_conducted                          0
search_vehicle                            0
search_basis                         851689
reason_for_stop                        2212
raw_search_vehicle_description            0
raw_result_of_cont

In [56]:
print("Percentage of missing values PER column:", "\n")
print((sf_stops_df.isna().mean() * 100))

Percentage of missing values PER column: 

raw_row_number                        0.000000
date                                  0.000000
time                                  0.003867
location                              0.004751
lat                                   0.187499
lng                                   0.187499
district                              5.766073
subject_age                           6.506458
subject_race                          0.000000
subject_sex                           0.000000
type                                  0.000000
arrest_made                           0.000000
citation_issued                       0.000000
warning_issued                        0.000000
outcome                               1.732573
contraband_found                     94.102003
search_conducted                      0.000000
search_vehicle                        0.000000
search_basis                         94.102003
reason_for_stop                       0.244401
raw_search_vehicl

___
### 1st Round of Narrowing Down Features:
#### Missing Values & Columns Related to "search_conducted"

* not considering "lat", "lng" since "location" can be used in place of those
* not considering "search_vehicle", "raw_search_vehicle_description" 
    * already have "search_conducted" (the variable trying to explain)
* not considering "outcome", "raw_result_of_contact_description", "citation_issued", "warning_issued", "arrest_made" 
    * these are also outcomes/results of the stop, but my goal is find features that *precede* whether or not a search was conducted
* looking at the percentage of missing values for each column above (excluding columns above ^^):
    * decide to not choose the top 2 columns w/ the highest percentages as features/predictors 
    * "contraband_found" and "search_basis" would be impossible to use w/ 94% of values missing

**columns not considered:** ["contraband_found", "search_basis", "lat", "lng",  "search_vehicle", "raw_search_vehicle_description", "outcome", "raw_result_of_contact_description", "citation_issued", "warning_issued", "arrest_made"]

In [95]:
sf_stops_drop1 = sf_stops_df.drop(["contraband_found", "search_basis", "lat", 
                                   "lng", "search_vehicle", "raw_search_vehicle_description", "outcome", 
                                   "raw_result_of_contact_description", "citation_issued", "warning_issued", "arrest_made"], 
                                   axis="columns")
sf_stops_drop1.head(3)

,raw_row_number,date,time,location,district,subject_age,subject_race,subject_sex,type,search_conducted,reason_for_stop
0,869921,2014-08-01,00:01:00,MASONIC AV & FELL ST,NaN,NaN,asian/pacific islander,female,vehicular,False,Mechanical or Non-Moving Violation (V.C.)
1,869922,2014-08-01,00:01:00,GEARY&10TH AV,NaN,NaN,black,male,vehicular,False,Mechanical or Non-Moving Violation (V.C.)
2,869923,2014-08-01,00:15:00,SUTTER N OCTAVIA ST,NaN,NaN,hispanic,male,vehicular,False,Mechanical or Non-Moving Violation (V.C.)


___
### 2nd Round of Narrowing Down Features: 
#### Assessing Variance

1st round: removed columns with too many missing values

**2nd round: need to find features that are actually useful for analysis**

* feature is only useful if it varies (important for modeling but also EDA) aka have reasonable spread across its categories
* why variance matters for plotting and insights:
    * if a feature has no variance (e.g 100% of the values are "male") -> plot is just single bar; no useful insights
    * also bad for bivariate analysis (the feature vs "search_conducted")
    * bivariate analysis: "examines the relationship between two variables to find correlations, dependencies, and interactions" (from Google)

In [96]:
remainder_cols = sf_stops_drop1.drop(["raw_row_number", "date", "time", "location", "search_conducted"], axis="columns").columns

# check the value counts of the remaining categorical columns
for col in remainder_cols:
    print(f"--- Value Counts for: {col} ---")
    print(sf_stops_drop1[col].value_counts(normalize=True).head(10))
    print("\n")

--- Value Counts for: district ---
district
H    0.137791
I    0.126879
B    0.124128
C    0.111863
D    0.102830
G    0.099986
E    0.086584
A    0.081696
F    0.072358
J    0.054135
Name: proportion, dtype: float64


--- Value Counts for: subject_age ---
subject_age
25.0    0.038949
30.0    0.035873
26.0    0.032896
27.0    0.032546
28.0    0.032309
24.0    0.032294
23.0    0.031044
35.0    0.029996
29.0    0.029600
22.0    0.028458
Name: proportion, dtype: float64


--- Value Counts for: subject_race ---
subject_race
white                     0.411369
asian/pacific islander    0.174223
black                     0.168159
hispanic                  0.128182
other                     0.118066
Name: proportion, dtype: float64


--- Value Counts for: subject_sex ---
subject_sex
male      0.706265
female    0.293735
Name: proportion, dtype: float64


--- Value Counts for: type ---
type
vehicular    1.0
Name: proportion, dtype: float64


--- Value Counts for: reason_for_stop ---
reason_for_

**will not consider "type"**

* "vehicular" makes up 100% of the "type" column -> no variance

**columns not considered:** ["contraband_found", "search_basis", "lat", "lng",  "search_vehicle", "raw_search_vehicle_description", "outcome", "raw_result_of_contact_description", "citation_issued", "warning_issued", "arrest_made", "type"]

___
### 3rd Round of Narrowing Down Features: 
#### Granularity

Remaining Columns: ['date', 'time', 'location', 'district, 'subject_age', 'subject_race', 'subject_sex', 'type']

* note: ignoring 'reason_for_stop' as a feature bc (recall that) will be controlling for it
* not considering "date" because it is too granular/specific
    * if time wasn't a constraint, I would engineer this into broader categories like day of the week for such feature to be useful
* not considering "location" bc it has 313002 unique values which won't be good for plotting or extracting insights
    * a better feature related to "location" would be "district"

**columns not considered:** ["contraband_found", "search_basis", "lat", "lng",  "search_vehicle", "raw_search_vehicle_description", "outcome", "raw_result_of_contact_description", "citation_issued", "warning_issued", "arrest_made", "type", "date", "location"]


In [99]:
sf_stops_drop_final = sf_stops_drop1.drop(["type", "date", "location"], axis="columns")
sf_stops_drop_final.head(3)

,raw_row_number,time,district,subject_age,subject_race,subject_sex,search_conducted,reason_for_stop
0,869921,00:01:00,NaN,NaN,asian/pacific islander,female,False,Mechanical or Non-Moving Violation (V.C.)
1,869922,00:01:00,NaN,NaN,black,male,False,Mechanical or Non-Moving Violation (V.C.)
2,869923,00:15:00,NaN,NaN,hispanic,male,False,Mechanical or Non-Moving Violation (V.C.)


In [102]:
print("Count of Unique Locations:", len(sf_stops_drop1["location"].unique()))

Count of Unique Locations: 313002


In [103]:
print("Unique Districts:", sf_stops_drop_final["district"].unique())

Unique Districts: [nan 'C' 'B' 'I' 'A' 'J' 'E' 'H' 'G' 'F' 'D' 'K' 'T' 'S']


In [104]:
sf_stops_drop_final.columns

Index(['raw_row_number', 'time', 'district', 'subject_age', 'subject_race',
       'subject_sex', 'search_conducted', 'reason_for_stop'],
      dtype='object')

___
##### My Chosen 5 Features:
['time', 'district', 'subject_age', 'subject_race', 'subject_sex']
___

### Other Data Quality Issue: 
#### Raw Data Inputted Weirdly
**1. The Finding:**
* found that 2,455 rows (approx 0.27% of the data) appear to be incorrectly inputted by the data collectors where multiple stops were merged into a single row
    
    ex: one row had a raw_row_number of "818282|818283|..." and a corresponding reason_for_stop of "Moving Violation|Mechanical..."

**2. What to do:** 
* these rows are ambiguous and unusable for analysis because a single row with various reason_for_stop values cannot be correctly categorized
* keeping them would introduce error and noise into any visual or statistical analysis
* action: since these bad quality rows only make up about 0.27% of the data, I will just drop them 
    * rather than trying to clean them as I'm not the data collector (might not clean up correctly)

In [125]:
print("percentage of bad quality rows:", sf_stops_drop_final["raw_row_number"].str.contains("|", regex=False).sum() / len(sf_stops_drop_final) * 100)

percentage of bad quality rows: 0.2712497375893577


In [126]:
# identify rows with '|' in "raw_row_number" column
bad_quality_data = sf_stops_drop_final[sf_stops_drop_final["raw_row_number"].str.contains("|", regex=False)]

display(bad_quality_data["reason_for_stop"].unique())
bad_quality_data.head(4)

array(['Mechanical or Non-Moving Violation (V.C.)',
       'Moving Violation|Mechanical or Non-Moving Violation (V.C.)',
       'Moving Violation', 'DUI Check|MPC Violation', 'Traffic Collision',
       'Moving Violation|NA', 'MPC Violation',
       'Mechanical or Non-Moving Violation (V.C.)|Moving Violation',
       'Moving Violation|Assistance to Motorist',
       'Moving Violation|DUI Check', 'Moving Violation|BOLO/APB/Warrant',
       'Assistance to Motorist', 'NA|Traffic Collision',
       'Mechanical or Non-Moving Violation (V.C.)|Assistance to Motorist',
       'Mechanical or Non-Moving Violation (V.C.)|DUI Check',
       'Moving Violation|MPC Violation',
       'Moving Violation|Mechanical or Non-Moving Violation (V.C.)|MPC Violation',
       'Moving Violation|MPC Violation|MPC Violation',
       'Moving Violation|Mechanical or Non-Moving Violation (V.C.)|Mechanical or Non-Moving Violation (V.C.)',
       'Moving Violation|Traffic Collision', 'BOLO/APB/Warrant',
       'Mechani

,raw_row_number,time,district,subject_age,subject_race,subject_sex,search_conducted,reason_for_stop
1008,855677|855678,22:20:00,NaN,NaN,other,male,False,Mechanical or Non-Moving Violation (V.C.)
1095,893069|893070,11:40:00,NaN,NaN,black,male,False,Moving Violation|Mechanical or Non-Moving Viol...
1147,893122|893123,15:28:00,NaN,NaN,black,female,False,Moving Violation|Mechanical or Non-Moving Viol...
2215,862668|862669,01:15:00,NaN,NaN,black,male,False,Mechanical or Non-Moving Violation (V.C.)


In [128]:
# get indexes of bad quality rows & remove them from final df
bad_quality_indexes = bad_quality_data.index.tolist()
sf_stops_clean = sf_stops_drop_final.drop(bad_quality_indexes, axis="index")

sf_stops_clean.head(3)

,raw_row_number,time,district,subject_age,subject_race,subject_sex,search_conducted,reason_for_stop
0,869921,00:01:00,NaN,NaN,asian/pacific islander,female,False,Mechanical or Non-Moving Violation (V.C.)
1,869922,00:01:00,NaN,NaN,black,male,False,Mechanical or Non-Moving Violation (V.C.)
2,869923,00:15:00,NaN,NaN,hispanic,male,False,Mechanical or Non-Moving Violation (V.C.)


In [ ]:
# check if removing bad quality rows worked 
# (should return array with single value for "reason_for_stop")
sf_stops_clean["reason_for_stop"].unique()

array(['Mechanical or Non-Moving Violation (V.C.)', 'Moving Violation',
       'MPC Violation', 'DUI Check', nan, 'Traffic Collision',
       'Assistance to Motorist', 'BOLO/APB/Warrant'], dtype=object)